In [10]:
from llama_index.core import (
    SimpleDirectoryReader,
    load_index_from_storage,
    VectorStoreIndex,
    StorageContext,
    ServiceContext,
)
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.llms.groq import Groq 
from llama_index.core.node_parser import SemanticSplitterNodeParser
from llama_index.postprocessor.cohere_rerank import CohereRerank
import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
COHERE_API_KEY = os.getenv("COHERE_API_KEY")

In [5]:
reader = SimpleDirectoryReader("data")
documents = reader.load_data()

In [13]:
embed_model = GeminiEmbedding(
    model = 'models/embeddings-001', api_key = GOOGLE_API_KEY
)
splitter = SemanticSplitterNodeParser(
    buffer_size=1, 
    breakpoint_percentile_threshold=95, 
    embed_model=embed_model
)
nodes = splitter.get_nodes_from_documents(documents, show_progress=True)

Parsing nodes: 100%|██████████| 135/135 [29:12<00:00, 12.98s/it]


In [26]:
llm = Groq(model="mixtral-8x7b-32768", api_key=GROQ_API_KEY)


In [31]:
service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=llm)

C:\Users\HP\AppData\Local\Temp\ipykernel_17500\59880135.py:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=llm)


In [32]:
vector_index = VectorStoreIndex.from_documents(documents, show_progress=True, 
               service_context=service_context, node_parser=nodes)

Generating embeddings: 100%|██████████| 135/135 [01:18<00:00,  1.72it/s]


In [33]:
vector_index.storage_context.persist(persist_dir="./storage")

In [34]:
storage_context = StorageContext.from_defaults(persist_dir="./storage")

In [35]:
index = load_index_from_storage(storage_context, service_context=service_context)

In [36]:
cohere_rerank = CohereRerank(api_key=COHERE_API_KEY, top_n=2)

In [37]:
query_engine = index.as_query_engine(service_context=service_context,
                similarity_top_k=10,
                node_postprocessors=[cohere_rerank],)

In [41]:
query = "Explain simple linear regression?"
query = query_engine.query(query)
print(query)

Linear regression is a type of regression algorithm that creates a model which is a linear combination of the input example's features. The goal is to find the optimal parameter values for the model so that it makes the most accurate predictions. The model's output is a real number, and it is used to predict the unknown target value for a given input example.

The model's form is similar to that of a Support Vector Machine (SVM) model, but instead of maximizing the distance between the hyperplane and the examples, the linear regression model's hyperplane is chosen to be as close to all training examples as possible. This is because the hyperplane is used to predict the value of the target for new, unlabeled input examples.

The optimization procedure used to find the optimal parameter values tries to minimize the sum of the squared differences between the model's output and the true target value for each training example. This expression is called the objective function, and the square